In [1]:
import os
import glob

import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from preprocessing import preprocessing, convert_spectrograms, convert_tensor
from model_ae import Encoder
from utils.optimization import WarmupLinearSchedule

In [2]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, num_attn_heads, attn_hidden_size, dropout_prob, with_focus_attn):
        super(MultiHeadedAttention, self).__init__()
        self.num_attn_heads = num_attn_heads
        self.hidden_size = attn_hidden_size
        self.dropout_prob = dropout_prob
        self.with_focus_attn = with_focus_attn
        
        self.attn_head_size = int(self.hidden_size / self.num_attn_heads)
        self.all_head_size = self.num_attn_heads * self.attn_head_size

        self.query = nn.Linear(self.hidden_size, self.all_head_size)
        self.key = nn.Linear(self.hidden_size, self.all_head_size)
        self.value = nn.Linear(self.hidden_size, self.all_head_size)

        self.o_proj = nn.Linear(self.hidden_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_prob)

        self.softmax = nn.Softmax(dim=-1)
        
        if(with_focus_attn == True):
            self.tanh = nn.Tanh()
            self.sigmoid = nn.Sigmoid()
            
            self.linear_focus_query = nn.Linear(num_attn_heads * self.attn_head_size, 
                                                num_attn_heads * self.attn_head_size)
            self.linear_focus_global = nn.Linear(num_attn_heads * self.attn_head_size, 
                                                 num_attn_heads * self.attn_head_size)
            
            up = torch.randn(num_attn_heads, 1, self.attn_head_size)
            self.up = Variable(up, requires_grad=True).cuda()
            torch.nn.init.xavier_uniform_(self.up)
            
            uz = torch.randn(num_attn_heads, 1, self.attn_head_size)
            self.uz = Variable(uz, requires_grad=True).cuda()
            torch.nn.init.xavier_uniform_(self.uz)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attn_heads, self.attn_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states):
        key_len = hidden_states.size(1)
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)
        
        if(self.with_focus_attn == True):
            glo = torch.mean(mixed_query_layer, dim=1, keepdim=True)
            
            c = self.tanh(self.linear_focus_query(mixed_query_layer) + self.linear_focus_global(glo))
            c = self.transpose_for_scores(c)
            
            p = c * self.up
            p = p.sum(3).squeeze()
            z = c * self.uz
            z = z.sum(3).squeeze()
            
            P = self.sigmoid(p) * key_len
            Z = self.sigmoid(z) * key_len
            
            j = torch.arange(start=0, end=key_len, dtype=P.dtype).unsqueeze(0).unsqueeze(0).unsqueeze(0).to('cuda')
            P = P.unsqueeze(-1)
            Z = Z.unsqueeze(-1)
            
            G = -(j - P)**2 * 2 / (Z**2)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attn_head_size)
        
        if(self.with_focus_attn == True):
            attention_scores = attention_scores + G
            
        attention_probs = self.softmax(attention_scores)
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        attention_output = self.o_proj(context_layer)

        return attention_output

class CLDNN(nn.Module):
    def __init__(self, conv_dim, checkpoint=None, hidden_size=60, num_layers=2,
                 bidirectional=True, with_focus_attn=False):
        super(CLDNN, self).__init__()
        self.conv_dim = conv_dim
        if(conv_dim == '2d'):
            self.conv1 = nn.Sequential(
                nn.Conv2d(1, 64, (3, 3)), # (1, 128, 50) -> (64, 126, 48)
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(3, 3)  # (64, 126, 48) -> (64, 42, 16)
            )
            self.conv2 = nn.Sequential(
                nn.Conv2d(64, 64, (3, 3)), # (64, 42, 16) -> (64, 40, 14)
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(3, 3)  # (64, 40, 14) -> (64, 13, 4)
            )
            self.lstm = nn.LSTM(832, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
            self.tanh = nn.Tanh()
            self.dropout = nn.Dropout(0.5)
            self.attn = MultiHeadedAttention(num_attn_heads=8, attn_hidden_size=120, dropout_prob=0.1,
                                             with_focus_attn=with_focus_attn)
            self.fc = nn.Sequential(
                nn.Linear(4*2*hidden_size if bidirectional else hidden_size, 4)
            )
        else:
            raise ValueError("Convolution dimension not found: %s" % (conv_dim))
            
    def forward(self, x):
        if(self.conv_dim == '2d'):
            out = self.conv1(x)  # (batch, 1, 128, 50) -> (batch, 64, 42, 16)
            out = self.conv2(out)  # (batch, 64, 42, 16) -> (batch, 64, 13, 4)
            new_out_shape = out.size()[:1] + (out.size()[1] * out.size()[2], out.size()[3])
            out = out.reshape(*new_out_shape)  # (batch, 64, 13, 4) -> (batch, 832, 4)
            out = out.permute(2, 0, 1)  # (batch, 832, 4) -> (4, batch, 832)
            self.lstm.flatten_parameters()
            out, _ = self.lstm(out)  # (4, batch, 832) -> (4, batch, 2*60)
            out = self.tanh(out)
            out = self.dropout(out)
            out = out.permute(1, 0, 2)  # (4, batch, 2*60) -> (batch, 4, 2*60)
            out = self.attn(out)  # (batch, 4, 2*60) -> (batch, 4, 2*60)
            new_out_shape = out.size()[:1] + (out.size()[1] * out.size()[2],)
            out = out.view(*new_out_shape)  # (batch, 4, 2*60) -> (batch, 4*2*60)
            out = self.fc(out)  # (batch, 4*2*60) -> (batch, 4)
        return out

In [3]:
# CNN -> LSTM -> self-attention -> DNN
class CLDNN(nn.Module):
    def __init__(self, conv_dim, checkpoint=None, hidden_size=64, num_layers=2,
                 bidirectional=True, with_focus_attn=False):
        super(CLDNN, self).__init__()
        self.conv_dim = conv_dim
        if(conv_dim == '1d'):
            self.conv1 = nn.Sequential(
                nn.Conv1d(1, 64, (3, 1)), # (1, 40, 50) -> (64, 38, 50)
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d((2, 1), (2, 1))  # (64, 38, 50) -> (64, 19, 50)
            )
            self.conv2 = nn.Sequential(
                nn.Conv1d(64, 64, (3, 1)), # (64, 19, 50) -> (64, 17, 50)
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d((2, 1), (2, 1))  # (64, 17, 50) -> (64, 8, 50)
            )
            self.lstm = nn.LSTM(512, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
            self.tanh = nn.Tanh()
            self.dropout = nn.Dropout(0.5)
            self.attn = MultiHeadedAttention(num_attn_heads=8, attn_hidden_size=128, dropout_prob=0.1,
                                             with_focus_attn=with_focus_attn)
            self.fc = nn.Sequential(
                nn.Linear(50*2*hidden_size if bidirectional else hidden_size, 128),
                nn.ReLU(),
                nn.Linear(128, 4),
                nn.Softmax()
            )
        else:
            raise ValueError("Convolution dimension not found: %s" % (conv_dim))
            
    def forward(self, x):
        if(self.conv_dim == '1d'):
            out = self.conv1(x)  # (batch, 1, 40, 50) -> (batch, 64, 19, 50)
            out = self.conv2(out)  # (batch, 64, 19, 50) -> (batch, 64, 8, 50)
            out = out.contiguous()
            new_out_shape = out.size()[:1] + (out.size()[1] * out.size()[2], out.size()[3])
            out = out.view(*new_out_shape)  # (batch, 64, 8, 50) -> (batch, 512, 50)
            out = out.permute(2, 0, 1)  # (batch, 512, 50) -> (50, batch, 512)
            self.lstm.flatten_parameters()
            out, _ = self.lstm(out)  # (50, batch, 512) -> (50, batch, 2*64)
            out = self.tanh(out)
            out = self.dropout(out)
            out = out.permute(1, 0, 2)  # (50, batch, 2*64) -> (batch, 50, 2*64)
            out = self.attn(out)  # (batch, 50, 2*64) -> (batch, 50, 2*64)
            new_out_shape = out.size()[:1] + (out.size()[1] * out.size()[2],)
            out = out.view(*new_out_shape)  # (batch, 50, 2*64) -> (batch, 50*2*64)
            out = self.fc(out)  # (batch, 50*2*64) -> (batch, 4)
        return out

# CNN -> self-attention -> LSTM -> DNN
class CLDNN(nn.Module):
    def __init__(self, conv_dim, checkpoint=None, hidden_size=64, num_layers=2,
                 bidirectional=True, with_focus_attn=False):
        super(CLDNN, self).__init__()
        self.conv_dim = conv_dim
        if(conv_dim == '1d'):
            self.conv1 = nn.Sequential(
                nn.Conv1d(1, 64, (3, 1)), # (1, 40, 50) -> (64, 38, 50)
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d((2, 1), (2, 1))  # (64, 38, 50) -> (64, 19, 50)
            )
            self.conv2 = nn.Sequential(
                nn.Conv1d(64, 64, (3, 1)), # (64, 19, 50) -> (64, 17, 50)
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d((2, 1), (2, 1))  # (64, 17, 50) -> (64, 8, 50)
            )
            self.attn = MultiHeadedAttention(num_attn_heads=8, attn_hidden_size=512, dropout_prob=0.1,
                                             with_focus_attn=with_focus_attn)
            self.lstm = nn.LSTM(512, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
            self.tanh = nn.Tanh()
            self.dropout = nn.Dropout(0.5)
            self.fc = nn.Sequential(
                nn.Linear(50*2*hidden_size if bidirectional else hidden_size, 128),
                nn.ReLU(),
                nn.Linear(128, 4)
            )
        else:
            raise ValueError("Convolution dimension not found: %s" % (conv_dim))
            
    def forward(self, x):
        if(self.conv_dim == '1d'):
            out = self.conv1(x)  # (batch, 1, 40, 50) -> (batch, 64, 19, 50)
            out = self.conv2(out)  # (batch, 64, 19, 50) -> (batch, 64, 8, 50)
            new_out_shape = out.size()[:1] + (out.size()[1] * out.size()[2], out.size()[3])
            out = out.reshape(*new_out_shape)  # (batch, 64, 8, 50) -> (batch, 512, 50)
            out = out.permute(0, 2, 1)  # (batch, 512, 50) -> (batch, 50, 512)
            h = out
            out = self.attn(out)  # (batch, 50, 512) -> (batch, 50, 512)
            out = h + out
            out = out.permute(1, 0, 2)  # (batch, 50, 512) -> (50, batch, 512)
            self.lstm.flatten_parameters()
            out, _ = self.lstm(out)  # (50, batch, 512) -> (50, batch, 2*64)
            #out = out[-1]  # (50, batch, 2*64) -> (batch, num_directions*hidden_size)
            out = self.tanh(out)
            out = self.dropout(out)
            out = out.permute(1, 0, 2)  # (50, batch, 2*64) -> (batch, 50, 2*64)
            new_out_shape = out.size()[:1] + (out.size()[1] * out.size()[2],)
            out = out.view(*new_out_shape)  # (batch, 50, 2*64) -> (batch, 50*2*64)       
            out = self.fc(out)  # (batch, 50*2*64) -> (batch, 4)
        return out

In [4]:
class CLDNN_G(nn.Module):
    def __init__(self, conv_dim, checkpoint=None, hidden_size=64, num_layers=2,
                 bidirectional=True, with_focus_attn=False):
        super(CLDNN_G, self).__init__()
        self.conv_dim = conv_dim
        if(conv_dim == '1d'):
            self.encoder = Encoder(conv_dim)
            if checkpoint:
                self.encoder.load_state_dict(torch.load(checkpoint))
            self.attn = MultiHeadedAttention(num_attn_heads=4, attn_hidden_size=8, dropout_prob=0.1,
                                             with_focus_attn=with_focus_attn)
            self.lstm = nn.LSTM(8, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
            self.fc = nn.Sequential(
                nn.Linear(hidden_size*2 if bidirectional else hidden_size, 1),
                nn.Sigmoid()
            )
        elif(conv_dim == '2d'):
            self.encoder = Encoder(conv_dim)
            if checkpoint:
                self.encoder.load_state_dict(torch.load(checkpoint))
            self.attn = MultiHeadedAttention(num_attn_heads=4, attn_hidden_size=176, dropout_prob=0.1, 
                                             with_focus_attn=with_focus_attn)
            self.gap = nn.AdaptiveAvgPool2d((1, 11))
            self.lstm = nn.LSTM(11, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
            self.fc = nn.Sequential(
                nn.Linear(hidden_size*2 if bidirectional else hidden_size, 1),
                nn.Sigmoid()
            )
        else:
            raise ValueError("Convolution dimension not found: %s" % (conv_dim))
            
    def forward(self, x):
        if(self.conv_dim == '1d'):
            out = self.encoder(x)  # (batch, 1, 40, 100) -> (batch, 8, 1, 100)
            out = torch.squeeze(out, 2)  # (batch, 8, 1, 100) -> (batch, 8, 100)
            out = out.permute(0, 2, 1)  # (batch, 8, 100) -> (batch, 100, 8)
            h = out
            out = self.attn(out)  # (batch, 100, 8) -> (batch, 100, 8)
            out = h + out
            out = out.permute(1, 0, 2)  # (batch, 100, 8) -> (100, batch, 8)
            self.lstm.flatten_parameters()
            out, _ = self.lstm(out)  # (100, batch, 8) -> (100, batch, num_directions*hidden_size)
            out = out[-1]  # (100, batch, num_directions*hidden_size) -> (batch, num_directions*hidden_size)
            out = self.fc(out)  # (batch, num_directions*hidden_size) -> (batch, 1)
        elif(self.conv_dim == '2d'):
            out = self.encoder(x)  # (batch, 1, 128, 100) -> (batch, 16, 11, 8)
            out = out.permute(0, 3, 1, 2)  # (batch, 16, 11, 8) -> (batch, 8, 16, 11)
            h = out
            new_out_shape = out.size()[:2] + (out.size()[2] * out.size()[3],)
            out = out.view(*new_out_shape)  # (batch, 8, 16, 11) -> (batch, 8, 176)
            out = self.attn(out)  # (batch, 8, 176) -> (batch, 8, 176)
            out = out.view(h.size())  # (batch, 8, 176) -> (batch, 8, 16, 11)
            out = h + out
            out = self.gap(out)  # (batch, 8, 16, 11) -> (batch, 8, 1, 11)
            out = torch.squeeze(out, 2)  # (batch, 8, 1, 11) -> (batch, 8, 11)
            out = out.permute(1, 0, 2)  # (batch, 8, 11) -> (8, batch, 11)
            self.lstm.flatten_parameters()
            out, _ = self.lstm(out)  # (8, batch, 11) -> (8, batch, num_directions*hidden_size)
            out = out[-1]  # (8, batch, num_directions*hidden_size) -> (batch, num_directions*hidden_size)
            out = self.fc(out)  # (batch, num_directions*hidden_size) -> (batch, 1)
        return out

In [5]:
def train(train_dataloader, eval_dataloader, epochs):
    print('Start training')
    max_acc = 0
    acc_list = []
    softmax = nn.Softmax(dim=1)
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        nb_train_steps = 0
        correct = 0
        num_samples = 0

        if(multi_task == 'true'):
            for X_batch, y_batch, y_g_batch in train_dataloader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                y_g_batch = y_g_batch.to(device)

                optimizer.zero_grad()

                outputs = model(X_batch)
                outputs_g = model_g(X_batch)

                loss_1 = loss_func(outputs, y_batch)
                loss_2 = loss_func_g(outputs_g, y_g_batch)
                loss = loss_1 + 0.8 * loss_2
                loss.backward(retain_graph=True)

                optimizer.step()
                opt_scheduler.step()

                train_loss += loss.mean().item()
                nb_train_steps += 1

                outputs = softmax(outputs)
                outputs = torch.argmax(outputs, dim=1)
                correct += (outputs == y_batch).float().sum()
                num_samples += len(X_batch)

            train_loss = train_loss / nb_train_steps
            train_accuracy = correct / num_samples

            model.eval()
            eval_loss = 0
            nb_eval_steps = 0
            correct = 0
            num_samples = 0

            for X_batch, y_batch, y_g_batch in eval_dataloader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                y_g_batch = y_g_batch.to(device)
                with torch.no_grad():
                    outputs = model(X_batch)
                    outputs_g = model_g(X_batch)

                tmp_eval_loss_1 = loss_func(outputs, y_batch)
                tmp_eval_loss_2 = loss_func_g(outputs_g, y_g_batch)
                tmp_eval_loss = tmp_eval_loss_1 + 0.8 * tmp_eval_loss_2
                eval_loss += tmp_eval_loss.mean().item()
                nb_eval_steps += 1

                outputs = softmax(outputs)
                outputs = torch.argmax(outputs, dim=1)
                correct += (outputs == y_batch).float().sum()
                num_samples += len(X_batch)

            eval_loss = eval_loss / nb_eval_steps
            eval_accuracy = correct / num_samples
        else:
            for X_batch, y_batch in train_dataloader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                optimizer.zero_grad()

                outputs = model(X_batch)

                loss = loss_func(outputs, y_batch)
                loss.backward()

                optimizer.step()
                if(use_warmup == 'true'):
                    opt_scheduler.step()

                train_loss += loss.mean().item()
                nb_train_steps += 1

                #outputs = softmax(outputs)
                #outputs = torch.argmax(outputs, dim=1)
                #correct += (outputs == y_batch).float().sum()
                #num_samples += len(X_batch)

            train_loss = train_loss / nb_train_steps
            #train_accuracy = correct / num_samples

            model.eval()
            eval_loss = 0
            nb_eval_steps = 0
            correct = 0
            num_samples = 0

            for X_batch, y_batch in eval_dataloader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                with torch.no_grad():
                    outputs = model(X_batch)

                tmp_eval_loss = loss_func(outputs, y_batch)
                eval_loss += tmp_eval_loss.mean().item()
                nb_eval_steps += 1

                #outputs = softmax(outputs)
                #outputs = torch.argmax(outputs, dim=1)
                #correct += (outputs == y_batch).float().sum()
                #num_samples += len(X_batch)

            eval_loss = eval_loss / nb_eval_steps
            #eval_accuracy = correct / num_samples

        for param_group in optimizer.param_groups:
            lr = param_group['lr']
        print('epoch: {:3d},    lr={:6f},    loss={:5f},    eval_loss={:5f}'
              .format(epoch+1, lr, train_loss, eval_loss))
        #print('epoch: {:3d},    lr={:6f},    loss={:5f},    train_acc={:5f},    eval_loss={:5f},    eval_acc={:5f}'
        #      .format(epoch+1, lr, train_loss, train_accuracy, eval_loss, eval_accuracy))

        if((epoch+1) % save_checkpoint_steps == 0):
            '''
            correct = 0
            n = 0
            for i in range(len(eval_samples)):
                try:
                    X_new = preprocessing(eval_samples[i], method='mfcc', sr=16000, n_mfcc=n_mfcc)
                    X_new = convert_tensor(X_new).to(device)
                    y_new = model(X_new)
                    y_new = torch.argmax(torch.mean(nn.Softmax(dim=-1)(y_new), dim=0))
                    #y_new = torch.argmax(nn.Softmax(dim=-1)(torch.mean(y_new, dim=0)))
                    #y_new = sorted(dict(collections.Counter(torch.argmax(nn.Softmax(dim=-1)(y_new), dim=1).cpu().numpy()))
                    #               .items(), key=(lambda x: x[1]), reverse=True)[0][0]
                    y_new = 1 if (y_new.item() == np.array(eval_label)[i]) else 0
                    correct += y_new
                    n += 1
                except:
                    pass
            acc = correct / n
            acc_list.append(acc)
            print('Test accuray:', round(acc, 5))
            '''
            
            model_checkpoint = "CLDNN_cv%d_step%d_epoch%d.pt" % (cv_iter, en+1, epoch+1)
            output_model_file = os.path.join(output_dir, model_checkpoint)
            torch.save(model.state_dict(), output_model_file)
            print("Saving checkpoint %s" % output_model_file)
            #if(acc > max_acc):
            #    max_acc =acc
            #    torch.save(model.state_dict(), output_model_file)
            #    print("Saving checkpoint %s" % output_model_file)
    #return max(acc_list)

In [6]:
focus_attn_list = [True]

In [7]:
df = pd.read_csv('IEMOCAP_sub_label.csv')
di = {'neu': 0, 'hap': 1, 'ang': 2, 'sad': 3}
df = df.replace({'sample_label': di})

In [8]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10)

In [9]:
label_path = './wav_data/pretrain/IEMOCAP_label/'

In [ ]:
for with_focus_attn in focus_attn_list:
    cv_eval = []
    cv_iter = 0
    #with_focus_attn = True if(with_focus_attn == 'true') else False
    for train_index, eval_index in skf.split(df['sample_name'], df['sample_label']):
        '''
    for ses_num in range(1, 6):
        eval_ses = 'Ses0' + str(ses_num)
        train_index = []
        eval_index = []
        for i, sample_name in enumerate(df['sample_name']):
            if eval_ses in sample_name:
                eval_index.append(i)
            else:
                train_index.append(i)
        '''
        cv_iter += 1
        conv_dim = '1d'
        checkpoint = ''
        hidden_size = 64
        num_layers = 2
        bidirectional = 'true'

        batch_size = 128
        num_epochs = 30
        learning_rate = 0.0001

        use_warmup = 'false'
        data_dir = './wav_data/pretrain/IEMOCAP_sub/'
        multi_task = 'false'
        augmentation = 'false'
        
        save_checkpoint_steps = 5
        output_dir = './model'

        bidirectional = True if(bidirectional == 'true') else False  
        n_mfcc = 40 if(conv_dim == '1d') else 128

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        train_samples, eval_samples = df['sample_name'][train_index], df['sample_name'][eval_index]
        train_label = [label_path + train_sample + '.npy' for train_sample in train_samples]
        eval_label = [label_path + eval_sample + '.npy' for eval_sample in eval_samples]
        
        train_samples = [data_dir + train_sample + '.wav' for train_sample in train_samples]
        eval_samples = [data_dir + eval_sample + '.wav' for eval_sample in eval_samples]

        train_label_list = []
        for i in range(len(train_label)):
            label = np.load(train_label[i])
            train_label_list.append(label)
        y_train = np.concatenate(train_label_list)

        eval_label_list = []
        for i in range(len(eval_label)):
            label = np.load(eval_label[i])
            eval_label_list.append(label)
        y_eval = np.concatenate(eval_label_list)

        if(multi_task == 'true'):
            speaker = np.array(list(map(lambda x: int(x.split('/')[-1].split('-')[-1].split('.')[0]), sample_datas)))
            y_gender = np.array(list(map(lambda x: 1 if x % 2 ==0 else 0, speaker)))

            y_g_train = y_gender[train_idx]
            y_g_eval = y_gender[eval_idx]

        X_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000)
        X_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000)

        if(augmentation == 'true'):
            X_train_flip = X_train[:, :, :, ::-1]
            y_train_flip = y_train.copy()

            X_train = np.concatenate((X_train, X_train_flip), axis=0)
            y_train = np.concatenate((y_train, y_train_flip), axis=0)

        X_train, y_train = convert_tensor(X_train, y_train)
        X_eval, y_eval = convert_tensor(X_eval, y_eval)

        #y_train = y_train.long()
        #y_eval = y_eval.long()

        if(multi_task == 'true'):
            _, y_g_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_g_train)
            _, y_g_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_g_eval)

            if(augmentation == 'true'):
                y_g_train_flip = y_g_train.copy()
                y_g_train = np.concatenate((y_g_train, y_g_train_flip))

            y_g_train = torch.tensor(y_g_train).float()
            y_g_eval = torch.tensor(y_g_eval).float()

            y_g_train = y_g_train.unsqueeze(-1)
            y_g_eval = y_g_eval.unsqueeze(-1)

        if(multi_task == 'true'):
            train_ds = TensorDataset(X_train, y_train, y_g_train)
            eval_ds = TensorDataset(X_eval, y_eval, y_g_eval)
        else:
            train_ds = TensorDataset(X_train, y_train)
            eval_ds = TensorDataset(X_eval, y_eval)

        train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
        eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, num_workers=0)

        for en in range(2):
            model = CLDNN(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                          num_layers=num_layers, bidirectional=bidirectional,
                          with_focus_attn=with_focus_attn).to(device)

            if(multi_task == 'true'):
                model_g = CLDNN_G(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                                    num_layers=num_layers, bidirectional=bidirectional,
                                    with_focus_attn=with_focus_attn).to(device)

            if(multi_task == 'true'):
                loss_func = nn.CrossEntropyLoss()
                loss_func_g = nn.BCELoss()
                optimizer = optim.Adam(list(model.parameters()) + list(model_g.parameters()), lr=learning_rate)
            else:
                loss_func = nn.KLDivLoss()
                optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            if(use_warmup == 'true'):
                t_total = len(train_dataloader) // 1 * num_epochs
                opt_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=t_total * 0.1, t_total=t_total)

            eval_acc = train(train_dataloader, eval_dataloader, num_epochs)
            cv_eval.append(eval_acc)

    print('conv_dim:', conv_dim, '\twith_focus_attn:', with_focus_attn)
    print('Test accuray:', cv_eval)
    print()

In [15]:
model_dir = './model/KLloss'
model_path_list = glob.glob(os.path.join(model_dir, '*pt'), recursive=True)

In [16]:
model_path_list

['./model/KLloss/CLDNN_cv1_step1_epoch5.pt',
 './model/KLloss/CLDNN_cv1_step1_epoch10.pt',
 './model/KLloss/CLDNN_cv1_step1_epoch15.pt',
 './model/KLloss/CLDNN_cv1_step1_epoch25.pt',
 './model/KLloss/CLDNN_cv1_step1_epoch30.pt',
 './model/KLloss/CLDNN_cv1_step1_epoch20.pt',
 './model/KLloss/CLDNN_cv1_step2_epoch5.pt',
 './model/KLloss/CLDNN_cv1_step2_epoch10.pt',
 './model/KLloss/CLDNN_cv1_step2_epoch15.pt',
 './model/KLloss/CLDNN_cv1_step2_epoch20.pt',
 './model/KLloss/CLDNN_cv1_step2_epoch25.pt',
 './model/KLloss/CLDNN_cv1_step2_epoch30.pt']

In [17]:
cv_iter = 0
for train_index, eval_index in skf.split(df['sample_name'], df['sample_label']):
    cv_iter += 1
    data_dir = './wav_data/pretrain/IEMOCAP_sub/'
    
    train_samples, eval_samples = df['sample_name'][train_index], df['sample_name'][eval_index]
    train_label, eval_label = df['sample_label'][train_index], df['sample_label'][eval_index]

    train_samples = [data_dir + train_sample + '.wav' for train_sample in train_samples]
    eval_samples = [data_dir + eval_sample + '.wav' for eval_sample in eval_samples]
    
    model_paths = [ckpt for ckpt in model_path_list if 'cv%d'%cv_iter in ckpt][:12]
    
    for ckpt in model_paths:
        model = CLDNN(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                              num_layers=num_layers, bidirectional=bidirectional,
                              with_focus_attn=with_focus_attn).to(device)
        model.load_state_dict(torch.load(ckpt))
        model.eval()

        correct = 0
        n = 0
        for i in tqdm(range(len(eval_samples))):
            try:
                X_new = preprocessing(eval_samples[i], method='mfcc', sr=16000, n_mfcc=n_mfcc)
                X_new = convert_tensor(X_new).to(device)
                y_new = model(X_new)
                y_new = torch.argmax(torch.mean(nn.Softmax(dim=-1)(y_new), dim=0))
                #y_new = torch.argmax(nn.Softmax(dim=-1)(torch.mean(y_new, dim=0)))
                #y_new = sorted(dict(collections.Counter(torch.argmax(nn.Softmax(dim=-1)(y_new), dim=1).cpu().numpy()))
                #               .items(), key=(lambda x: x[1]), reverse=True)[0][0]
                y_new = 1 if (y_new.item() == np.array(eval_label)[i]) else 0
                correct += y_new
                n += 1
            except:
                pass
        acc = correct / n
        
        print(ckpt.split('/')[-1], ':', acc)

  0%|          | 0/449 [00:00<?, ?it/s]/home/junghun/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
  2%|▏         | 8/449 [00:00<00:06, 73.30it/s]

CLDNN_cv1_step1_epoch5.pt : 0.5679287305122495


  2%|▏         | 8/449 [00:00<00:05, 75.45it/s]

CLDNN_cv1_step1_epoch10.pt : 0.621380846325167


  2%|▏         | 8/449 [00:00<00:05, 74.58it/s]

...

CLDNN_cv1_step2_epoch20.pt : 0.5946547884187082


  2%|▏         | 8/449 [00:00<00:06, 71.44it/s]

CLDNN_cv1_step2_epoch25.pt : 0.6102449888641426


100%|██████████| 449/449 [00:07<00:00, 56.84it/s]

CLDNN_cv1_step2_epoch30.pt : 0.5679287305122495


In [10]:
with_focus_attn = [0.6570155902004454, 0.6124721603563474, 0.6347438752783965, 
                   0.6926503340757239, 0.6280623608017817, 0.6169265033407573, 
                   0.5812917594654788, 0.6013363028953229, 0.6948775055679287, 0.6792873051224945]
without_focus_attn = [0.6347438752783965, 0.6258351893095768, 0.6057906458797327, 
                      0.7037861915367484, 0.6035634743875279, 0.5946547884187082, 
                      0.6146993318485523, 0.6325167037861915, 0.688195991091314, 0.6681514476614699]

In [11]:
print('with_focus_attn:', sum(with_focus_attn) / 10)
print('without_focus_attn:', sum(without_focus_attn) / 10)

with_focus_attn: 0.6398663697104678
without_focus_attn: 0.6371937639198219


In [12]:
sum([p.numel() for p in model.parameters()])

1327044

In [13]:
model

CLDNN(
  (conv1): Sequential(
    (0): Conv1d(1, 64, kernel_size=(3, 1), stride=(1,))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv1d(64, 64, kernel_size=(3, 1), stride=(1,))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(512, 64, num_layers=2, bidirectional=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.5, inplace=False)
  (attn): MultiHeadedAttention(
    (query): Linear(in_features=128, out_features=128, bias=True)
    (key): Linear(in_features=128, out_features=128, bias=True)
    (value): Linear(in_features=128, out_features=128, bias=True)
    (o_proj): Linear(in_features=128, out_features=128, bias=True)
    (dropout): 

In [12]:
with_focus_attn = [0.623608017817372, 0.6503340757238307, 0.6280623608017817, 
                   0.7015590200445434, 0.6302895322939867, 0.6057906458797327, 
                   0.6057906458797327, 0.6102449888641426, 0.6547884187082406, 0.6570155902004454]
without_focus_attn = [0.6302895322939867, 0.6391982182628062, 0.623608017817372, 
                      0.7082405345211581, 0.6124721603563474, 0.6035634743875279, 
                      0.6102449888641426, 0.6124721603563474, 0.6547884187082406, 0.6636971046770601]

In [13]:
print('with_focus_attn:', sum(with_focus_attn) / 10)
print('without_focus_attn:', sum(without_focus_attn) / 10)

with_focus_attn: 0.6367483296213808
without_focus_attn: 0.6358574610244988


In [10]:
class CLDNN(nn.Module):
    def __init__(self, conv_dim, checkpoint=None, hidden_size=128, num_layers=2,
                 bidirectional=True, with_focus_attn=False):
        super(CLDNN, self).__init__()
        self.conv_dim = conv_dim
        if(conv_dim == '1d'):
            self.encoder = Encoder(conv_dim)
            if checkpoint:
                self.encoder.load_state_dict(torch.load(checkpoint))
            self.lstm = nn.LSTM(8, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
            self.tanh = nn.Tanh()
            self.dropout = nn.Dropout(0.1)
            self.attn = MultiHeadedAttention(num_attn_heads=4, attn_hidden_size=256, dropout_prob=0.1,
                                             with_focus_attn=with_focus_attn)
            self.fc = nn.Sequential(
                nn.Linear(50*2*hidden_size if bidirectional else 50*hidden_size, 64),
                nn.ReLU(),
                nn.Linear(64, 4)
            )
        elif(conv_dim == '2d'):
            self.encoder = Encoder(conv_dim)
            if checkpoint:
                self.encoder.load_state_dict(torch.load(checkpoint))
            self.attn = MultiHeadedAttention(num_attn_heads=4, attn_hidden_size=176, dropout_prob=0.1, 
                                             with_focus_attn=with_focus_attn)
            self.gap = nn.AdaptiveAvgPool2d((1, 11))
            self.lstm = nn.LSTM(11, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional) 
            self.fc = nn.Sequential(
                nn.Linear(hidden_size*2 if bidirectional else hidden_size, 4)
            )
        else:
            raise ValueError("Convolution dimension not found: %s" % (conv_dim))
            
    def forward(self, x):
        if(self.conv_dim == '1d'):
            out = self.encoder(x)  # (batch, 1, 40, 50) -> (batch, 8, 1, 50)
            out = torch.squeeze(out, 2)  # (batch, 8, 1, 50) -> (batch, 8, 50)
            out = out.permute(2, 0, 1)  # (batch, 8, 50) -> (50, batch, 8)
            out, _ = self.lstm(out)  # (50, batch, 8) -> (50, batch, num_directions*hidden_size)
            out = self.tanh(out)
            out = self.dropout(out)
            out = out.permute(1, 0, 2)  # (50, batch, num_directions*hidden_size) -> (batch, 50, num_directions*hidden_size)
            h = out
            out = self.attn(out) # (batch, 50, num_directions*hidden_size) -> (batch, 50, num_directions*hidden_size)
            out = h + out
            new_out_shape = out.size()[:1] + (out.size()[1] * out.size()[2],)
            out = out.reshape(*new_out_shape)  # (batch, 50, num_directions*hidden_size) -> (batch, 50*num_directions*hidden_size)
            out = self.fc(out)  # (batch, 50*num_directions*hidden_size) -> (batch, 4)
        elif(self.conv_dim == '2d'):
            out = self.encoder(x)  # (batch, 1, 128, 100) -> (batch, 16, 11, 8)
            out = out.permute(0, 3, 1, 2)  # (batch, 16, 11, 8) -> (batch, 8, 16, 11)
            h = out
            new_out_shape = out.size()[:2] + (out.size()[2] * out.size()[3],)
            out = out.view(*new_out_shape)  # (batch, 8, 16, 11) -> (batch, 8, 176)
            out = self.attn(out)  # (batch, 8, 176) -> (batch, 8, 176)
            out = out.view(h.size())  # (batch, 8, 176) -> (batch, 8, 16, 11)
            out = h + out
            out = self.gap(out)  # (batch, 8, 16, 11) -> (batch, 8, 1, 11)
            out = torch.squeeze(out, 2)  # (batch, 8, 1, 11) -> (batch, 8, 11)
            out = out.permute(1, 0, 2)  # (batch, 8, 11) -> (8, batch, 11)
            out, _ = self.lstm(out)  # (8, batch, 11) -> (8, batch, num_directions*hidden_size)
            out = out[-1]  # (8, batch, num_directions*hidden_size) -> (batch, num_directions*hidden_size)
            out = self.fc(out)  # (batch, num_directions*hidden_size) -> (batch, 1)
        return out

In [11]:
for with_focus_attn in focus_attn_list:
    cv_eval = []
    #with_focus_attn = True if(with_focus_attn == 'true') else False
    for train_index, eval_index in skf.split(df['sample_name'], df['sample_label']):
        conv_dim = '1d'
        checkpoint = ''
        hidden_size = 128
        num_layers = 2
        bidirectional = 'true'

        batch_size = 256
        num_epochs = 50
        learning_rate = 0.001

        use_warmup = 'true'
        data_dir = './wav_data/pretrain/IEMOCAP_sub/'
        multi_task = 'false'
        augmentation = 'false'
        
        save_checkpoint_steps = 5
        output_dir = './model'

        bidirectional = True if(bidirectional == 'true') else False   
        n_mfcc = 40 if(conv_dim == '1d') else 128

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        train_samples, eval_samples = df['sample_name'][train_index], df['sample_name'][eval_index]
        train_label, eval_label = df['sample_label'][train_index], df['sample_label'][eval_index]
        
        train_samples = [data_dir + train_sample + '.wav' for train_sample in train_samples]
        eval_samples = [data_dir + eval_sample + '.wav' for eval_sample in eval_samples]

        y_train = np.array(train_label)
        y_eval = np.array(eval_label)

        if(multi_task == 'true'):
            speaker = np.array(list(map(lambda x: int(x.split('/')[-1].split('-')[-1].split('.')[0]), sample_datas)))
            y_gender = np.array(list(map(lambda x: 1 if x % 2 ==0 else 0, speaker)))

            y_g_train = y_gender[train_idx]
            y_g_eval = y_gender[eval_idx]

        X_train, y_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_train)
        X_eval, y_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_eval)

        if(augmentation == 'true'):
            X_train_flip = X_train[:, :, :, ::-1]
            y_train_flip = y_train.copy()

            X_train = np.concatenate((X_train, X_train_flip), axis=0)
            y_train = np.concatenate((y_train, y_train_flip), axis=0)

        X_train, y_train = convert_tensor(X_train, y_train)
        X_eval, y_eval = convert_tensor(X_eval, y_eval)

        y_train = y_train.long()
        y_eval = y_eval.long()

        if(multi_task == 'true'):
            _, y_g_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_g_train)
            _, y_g_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_g_eval)

            if(augmentation == 'true'):
                y_g_train_flip = y_g_train.copy()
                y_g_train = np.concatenate((y_g_train, y_g_train_flip))

            y_g_train = torch.tensor(y_g_train).float()
            y_g_eval = torch.tensor(y_g_eval).float()

            y_g_train = y_g_train.unsqueeze(-1)
            y_g_eval = y_g_eval.unsqueeze(-1)

        if(multi_task == 'true'):
            train_ds = TensorDataset(X_train, y_train, y_g_train)
            eval_ds = TensorDataset(X_eval, y_eval, y_g_eval)
        else:
            train_ds = TensorDataset(X_train, y_train)
            eval_ds = TensorDataset(X_eval, y_eval)

        train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
        eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, num_workers=0)

        model = CLDNN(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                      num_layers=num_layers, bidirectional=bidirectional,
                      with_focus_attn=with_focus_attn).to(device)

        if(multi_task == 'true'):
            model_g = CLDNN_G(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                                num_layers=num_layers, bidirectional=bidirectional,
                                with_focus_attn=with_focus_attn).to(device)

        if(multi_task == 'true'):
            loss_func = nn.CrossEntropyLoss()
            loss_func_g = nn.BCELoss()
            optimizer = optim.Adam(list(model.parameters()) + list(model_g.parameters()), lr=learning_rate)
        else:
            loss_func = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        if(use_warmup == 'true'):
            t_total = len(train_dataloader) // 1 * num_epochs
            opt_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=t_total * 0.1, t_total=t_total)

        eval_acc = train(train_dataloader, eval_dataloader, num_epochs)
        cv_eval.append(eval_acc)

    print('conv_dim:', conv_dim, '\twith_focus_attn:', with_focus_attn)
    print('Test accuray:', cv_eval)
    print()

4041it [00:28, 141.44it/s]
449it [00:03, 139.56it/s]


Start training
epoch:   1,    lr=0.000200,    loss=1.145764,    train_acc=0.494389,    eval_loss=0.975452,    eval_acc=0.564436
epoch:   2,    lr=0.000400,    loss=0.999604,    train_acc=0.592781,    eval_loss=0.927232,    eval_acc=0.603390
epoch:   3,    lr=0.000600,    loss=0.949947,    train_acc=0.618561,    eval_loss=0.928751,    eval_acc=0.599315
epoch:   4,    lr=0.000800,    loss=0.923217,    train_acc=0.631525,    eval_loss=0.895027,    eval_acc=0.618005
epoch:   5,    lr=0.001000,    loss=0.904673,    train_acc=0.640466,    eval_loss=0.982887,    eval_acc=0.582636
Test accuray: 0.57238
epoch:   6,    lr=0.000978,    loss=0.889310,    train_acc=0.646794,    eval_loss=0.904333,    eval_acc=0.623492
epoch:   7,    lr=0.000956,    loss=0.874368,    train_acc=0.653000,    eval_loss=0.917226,    eval_acc=0.616212
epoch:   8,    lr=0.000933,    loss=0.860490,    train_acc=0.658562,    eval_loss=0.890120,    eval_acc=0.616321
epoch:   9,    lr=0.000911,    loss=0.846037,    train_acc=

10it [00:00, 97.14it/s]

Test accuray: 0.61024


4041it [00:26, 150.54it/s]
449it [00:02, 152.42it/s]


...

Start training
epoch:   1,    lr=0.000200,    loss=1.129147,    train_acc=0.500917,    eval_loss=1.048761,    eval_acc=0.542522
epoch:   2,    lr=0.000400,    loss=0.990143,    train_acc=0.591836,    eval_loss=0.998643,    eval_acc=0.586577
epoch:   3,    lr=0.000600,    loss=0.948823,    train_acc=0.612856,    eval_loss=1.007501,    eval_acc=0.586046
epoch:   4,    lr=0.000800,    loss=0.919298,    train_acc=0.627770,    eval_loss=0.960853,    eval_acc=0.617363
epoch:   5,    lr=0.001000,    loss=0.899950,    train_acc=0.636294,    eval_loss=0.987752,    eval_acc=0.598726
Test accuray: 0.64588
epoch:   6,    lr=0.000978,    loss=0.883561,    train_acc=0.645655,    eval_loss=0.972180,    eval_acc=0.604270
epoch:   7,    lr=0.000956,    loss=0.869533,    train_acc=0.651250,    eval_loss=1.003222,    eval_acc=0.589290
epoch:   8,    lr=0.000933,    loss=0.854317,    train_acc=0.657398,    eval_loss=1.054475,    eval_acc=0.581623
epoch:   9,    lr=0.000911,    loss=0.842359,    train_acc=

18it [00:00, 175.68it/s]

Test accuray: 0.61024


4041it [00:26, 152.21it/s]
449it [00:02, 161.13it/s]


Start training
epoch:   1,    lr=0.000200,    loss=1.125520,    train_acc=0.500184,    eval_loss=1.126347,    eval_acc=0.528246
epoch:   2,    lr=0.000400,    loss=0.989149,    train_acc=0.594161,    eval_loss=1.145453,    eval_acc=0.553721
epoch:   3,    lr=0.000600,    loss=0.942636,    train_acc=0.618543,    eval_loss=1.076926,    eval_acc=0.560239
epoch:   4,    lr=0.000800,    loss=0.914368,    train_acc=0.632996,    eval_loss=1.045484,    eval_acc=0.565345
epoch:   5,    lr=0.001000,    loss=0.891412,    train_acc=0.645114,    eval_loss=1.101681,    eval_acc=0.555894
Test accuray: 0.63697
epoch:   6,    lr=0.000978,    loss=0.875303,    train_acc=0.652442,    eval_loss=1.284294,    eval_acc=0.504834
epoch:   7,    lr=0.000956,    loss=0.859439,    train_acc=0.658648,    eval_loss=1.131919,    eval_acc=0.558501
epoch:   8,    lr=0.000933,    loss=0.844609,    train_acc=0.665835,    eval_loss=1.104000,    eval_acc=0.566105
epoch:   9,    lr=0.000911,    loss=0.831736,    train_acc=

In [14]:
with_focus_attn = [0.6146993318485523, 0.6035634743875279, 0.6169265033407573, 
                   0.6948775055679287, 0.6169265033407573, 0.5991091314031181, 
                   0.5946547884187082, 0.6080178173719376, 0.6391982182628062, 0.6592427616926503]
without_focus_attn = [0.6080178173719376, 0.6347438752783965, 0.6191536748329621, 
                      0.6815144766146993, 0.6035634743875279, 0.5835189309576837, 
                      0.5902004454342984, 0.5746102449888641, 0.6458797327394209, 0.6503340757238307]

In [15]:
print('with_focus_attn:', sum(with_focus_attn) / 10)
print('without_focus_attn:', sum(without_focus_attn) / 10)

with_focus_attn: 0.6247216035634744
without_focus_attn: 0.6191536748329621


In [ ]:
# 0.70824

In [10]:
with_focus_attn = [0.6414253897550112, 0.6280623608017817, 0.6169265033407573, 
                   0.6904231625835189, 0.6325167037861915, 0.6458797327394209, 
                   0.6035634743875279, 0.5991091314031181, 0.6614699331848553, 0.6503340757238307]

without_focus_attn = [0.6191536748329621, 0.6080178173719376, 0.6347438752783965, 
                      0.6859688195991092, 0.6302895322939867, 0.6057906458797327, 
                      0.6035634743875279, 0.6102449888641426, 0.6592427616926503, 0.6525612472160356]

In [11]:
print('with_focus_attn:', sum(with_focus_attn) / 10)
print('without_focus_attn:', sum(without_focus_attn) / 10)

with_focus_attn: 0.6369710467706013
without_focus_attn: 0.6309576837416481


In [9]:
for with_focus_attn in focus_attn_list:
    cv_eval = []
    #with_focus_attn = True if(with_focus_attn == 'true') else False
    for train_index, eval_index in skf.split(df['sample_name'], df['sample_label']):
        conv_dim = '1d'
        checkpoint = ''
        hidden_size = 128
        num_layers = 2
        bidirectional = 'true'

        batch_size = 256
        num_epochs = 50
        learning_rate = 0.001

        use_warmup = 'true'
        data_dir = './wav_data/pretrain/IEMOCAP_sub/'
        multi_task = 'false'
        augmentation = 'false'
        
        save_checkpoint_steps = 5
        output_dir = './model'
        
        bidirectional = True if(bidirectional == 'true') else False
        n_mfcc = 40 if(conv_dim == '1d') else 128

        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        
        train_samples, eval_samples = df['sample_name'][train_index], df['sample_name'][eval_index]
        train_label, eval_label = df['sample_label'][train_index], df['sample_label'][eval_index]
        
        train_samples = [data_dir + train_sample + '.wav' for train_sample in train_samples]
        eval_samples = [data_dir + eval_sample + '.wav' for eval_sample in eval_samples]

        y_train = np.array(train_label)
        y_eval = np.array(eval_label)

        if(multi_task == 'true'):
            speaker = np.array(list(map(lambda x: int(x.split('/')[-1].split('-')[-1].split('.')[0]), sample_datas)))
            y_gender = np.array(list(map(lambda x: 1 if x % 2 ==0 else 0, speaker)))

            y_g_train = y_gender[train_idx]
            y_g_eval = y_gender[eval_idx]

        X_train, y_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_train)
        X_eval, y_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_eval)

        if(augmentation == 'true'):
            X_train_flip = X_train[:, :, :, ::-1]
            y_train_flip = y_train.copy()

            X_train = np.concatenate((X_train, X_train_flip), axis=0)
            y_train = np.concatenate((y_train, y_train_flip), axis=0)

        X_train, y_train = convert_tensor(X_train, y_train)
        X_eval, y_eval = convert_tensor(X_eval, y_eval)

        y_train = y_train.long()
        y_eval = y_eval.long()

        if(multi_task == 'true'):
            _, y_g_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_g_train)
            _, y_g_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_g_eval)

            if(augmentation == 'true'):
                y_g_train_flip = y_g_train.copy()
                y_g_train = np.concatenate((y_g_train, y_g_train_flip))

            y_g_train = torch.tensor(y_g_train).float()
            y_g_eval = torch.tensor(y_g_eval).float()

            y_g_train = y_g_train.unsqueeze(-1)
            y_g_eval = y_g_eval.unsqueeze(-1)

        if(multi_task == 'true'):
            train_ds = TensorDataset(X_train, y_train, y_g_train)
            eval_ds = TensorDataset(X_eval, y_eval, y_g_eval)
        else:
            train_ds = TensorDataset(X_train, y_train)
            eval_ds = TensorDataset(X_eval, y_eval)

        train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
        eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, num_workers=0)

        model = CLDNN(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                      num_layers=num_layers, bidirectional=bidirectional,
                      with_focus_attn=with_focus_attn).to(device)

        if(multi_task == 'true'):
            model_g = CLDNN_G(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                                num_layers=num_layers, bidirectional=bidirectional,
                                with_focus_attn=with_focus_attn).to(device)

        if(multi_task == 'true'):
            loss_func = nn.CrossEntropyLoss()
            loss_func_g = nn.BCELoss()
            optimizer = optim.Adam(list(model.parameters()) + list(model_g.parameters()), lr=learning_rate)
        else:
            loss_func = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        if(use_warmup == 'true'):
            t_total = len(train_dataloader) // 1 * num_epochs
            opt_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=t_total * 0.1, t_total=t_total)

        eval_acc = train(train_dataloader, eval_dataloader, num_epochs)
        cv_eval.append(eval_acc)

    print('conv_dim:', conv_dim, '\twith_focus_attn:', with_focus_attn)
    print('Test accuray:', cv_eval)
    print()

4041it [00:25, 158.41it/s]
449it [00:03, 146.35it/s]


Start training
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 

c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch

glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 tor

glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 tor

c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch

P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Size([1, 1, 1, 50])
G torch.Size([256, 4, 50, 50])
glo torch.Size([256, 1, 8])
c_1 torch.Size([256, 50, 8])
c_2 torch.Size([256, 4, 50, 2])
up torch.Size([4, 1, 2])
p torch.Size([256, 4, 50, 2])
uz torch.Size([4, 1, 2])
z torch.Size([256, 4, 50, 2])
P torch.Size([256, 4, 50])
Z torch.Size([256, 4, 50])
j torch.Si

KeyboardInterrupt: 

In [10]:
with_focus_attn

False

In [11]:
for with_focus_attn in focus_attn_list:
    print(with_focus_attn)

True
False


In [ ]:
# window_size 25ms, hop_size 10ms
for conv_dim in conv_dim_list:
    for augmentation in augmentation_list:
        for with_focus_attn in focus_attn_list:
            for multi_task in multi_task_list:
                #conv_dim = '1d'
                checkpoint = './output/aae_' + conv_dim + '_step_500.pt'
                hidden_size = 128
                num_layers = 2
                bidirectional = 'true'
                #with_focus_attn = 'false'

                batch_size = 128
                num_epochs = 300
                learning_rate = 0.0001

                use_warmup = 'true'
                data_dir = './wav_data/pretrain/RAVDESS_resample/'
                #multi_task = 'false'
                #augmentation = 'true'

                bidirectional = True if(bidirectional == 'true') else False
                #with_focus_attn = True if(with_focus_attn == 'true') else False
                n_mfcc = 40 if(conv_dim == '1d') else 128

                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

                sample_datas = glob.glob(os.path.join(data_dir, '**', '*wav'), recursive=True)
                sample_datas = sorted(sample_datas)

                acc_list = []
                for i in range(5):
                    np.random.seed(10 * i + 3)
                    idx = np.random.permutation(len(sample_datas))
                    train_idx = idx[:int(len(sample_datas)*0.75)]
                    eval_idx = idx[int(len(sample_datas)*0.75):]

                    train_samples = list(np.array(sample_datas)[train_idx])
                    eval_samples = list(np.array(sample_datas)[eval_idx])

                    y = np.array(list(map(lambda x: int(x.split('/')[-1].split('-')[2]) - 1, sample_datas)))
                    y_train = y[train_idx]
                    y_eval = y[eval_idx]

                    if(multi_task == 'true'):
                        speaker = np.array(list(map(lambda x: int(x.split('/')[-1].split('-')[-1].split('.')[0]), sample_datas)))
                        y_gender = np.array(list(map(lambda x: 1 if x % 2 ==0 else 0, speaker)))

                        y_g_train = y_gender[train_idx]
                        y_g_eval = y_gender[eval_idx]

                    X_train, y_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_train)
                    X_eval, y_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_eval)

                    if(augmentation == 'true'):
                        X_train_flip = X_train[:, :, :, ::-1]
                        y_train_flip = y_train.copy()

                        X_train = np.concatenate((X_train, X_train_flip), axis=0)
                        y_train = np.concatenate((y_train, y_train_flip), axis=0)
                    
                    X_train, y_train = convert_tensor(X_train, y_train)
                    X_eval, y_eval = convert_tensor(X_eval, y_eval)

                    y_train = y_train.long()
                    y_eval = y_eval.long()

                    if(multi_task == 'true'):
                        _, y_g_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_g_train)
                        _, y_g_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_g_eval)

                        if(augmentation == 'true'):
                            y_g_train_flip = y_g_train.copy()
                            y_g_train = np.concatenate((y_g_train, y_g_train_flip))
                        
                        y_g_train = torch.tensor(y_g_train).float()
                        y_g_eval = torch.tensor(y_g_eval).float()

                        y_g_train = y_g_train.unsqueeze(-1)
                        y_g_eval = y_g_eval.unsqueeze(-1)

                    if(multi_task == 'true'):
                        train_ds = TensorDataset(X_train, y_train, y_g_train)
                        eval_ds = TensorDataset(X_eval, y_eval, y_g_eval)
                    else:
                        train_ds = TensorDataset(X_train, y_train)
                        eval_ds = TensorDataset(X_eval, y_eval)

                    train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
                    eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, num_workers=0, drop_last=True)

                    model = CLDNN(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                                  num_layers=num_layers, bidirectional=bidirectional,
                                  with_focus_attn=with_focus_attn).to(device)

                    if(multi_task == 'true'):
                        model_g = CLDNN_G(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                                            num_layers=num_layers, bidirectional=bidirectional,
                                            with_focus_attn=with_focus_attn).to(device)

                    if(multi_task == 'true'):
                        loss_func = nn.CrossEntropyLoss()
                        loss_func_g = nn.BCELoss()
                        optimizer = optim.Adam(list(model.parameters()) + list(model_g.parameters()), lr=learning_rate)
                    else:
                        loss_func = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                    if(use_warmup == 'true'):
                        t_total = len(train_dataloader) // 1 * num_epochs
                        opt_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=t_total * 0.1, t_total=t_total)

                    train(train_dataloader, eval_dataloader, num_epochs)

                    model.eval()
                    if(multi_task == 'true'):
                        model_g.eval()

                    correct = 0
                    n = 0
                    for i in range(len(eval_samples)):
                        try:
                            X_new = preprocessing(eval_samples[i], method='mfcc', sr=16000, n_mfcc=n_mfcc)
                            X_new = convert_tensor(X_new).to(device)
                            y_new = model(X_new)
                            y_new = torch.argmax(nn.Softmax(dim=-1)(torch.mean(y_new, dim=0)))
                            #y_new = sorted(dict(collections.Counter(torch.argmax(nn.Softmax(dim=-1)(y_new), dim=1).cpu().numpy()))
                            #               .items(), key=(lambda x: x[1]), reverse=True)[0][0]
                            y_new = 1 if (y_new.item() == y[eval_idx][i].item()) else 0
                            correct += y_new
                            n += 1
                        except:
                            pass

                    acc = correct / n
                    acc_list.append(acc)

                acc_mean = sum(acc_list) / 5
                print('conv_dim:', conv_dim, '\taugmentation', augmentation, 
                      '\twith_focus_attn:', with_focus_attn, '\tmulti_task:', multi_task)
                print('Test accuray:', round(acc_mean, 5))
                print()

In [ ]:
for conv_dim in conv_dim_list:
    for augmentation in augmentation_list:
        for with_focus_attn in focus_attn_list:
            for multi_task in multi_task_list:
                #conv_dim = '1d'
                checkpoint = './output/aae_' + conv_dim + '_step_300.pt'
                hidden_size = 128
                num_layers = 2
                bidirectional = 'true'
                #with_focus_attn = 'false'

                batch_size = 128
                num_epochs = 300
                learning_rate = 0.0001

                use_warmup = 'true'
                data_dir = './wav_data/pretrain/RAVDESS_resample/'
                #multi_task = 'false'
                #augmentation = 'true'

                bidirectional = True if(bidirectional == 'true') else False
                #with_focus_attn = True if(with_focus_attn == 'true') else False
                n_mfcc = 40 if(conv_dim == '1d') else 128

                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

                sample_datas = glob.glob(os.path.join(data_dir, '**', '*wav'), recursive=True)
                sample_datas = sorted(sample_datas)

                acc_list = []
                for i in range(5):
                    np.random.seed(10 * i + 3)
                    idx = np.random.permutation(len(sample_datas))
                    train_idx = idx[:int(len(sample_datas)*0.75)]
                    eval_idx = idx[int(len(sample_datas)*0.75):]

                    train_samples = list(np.array(sample_datas)[train_idx])
                    eval_samples = list(np.array(sample_datas)[eval_idx])

                    y = np.array(list(map(lambda x: int(x.split('/')[-1].split('-')[2]) - 1, sample_datas)))
                    y_train = y[train_idx]
                    y_eval = y[eval_idx]

                    if(multi_task == 'true'):
                        speaker = np.array(list(map(lambda x: int(x.split('/')[-1].split('-')[-1].split('.')[0]), sample_datas)))
                        y_gender = np.array(list(map(lambda x: 1 if x % 2 ==0 else 0, speaker)))

                        y_g_train = y_gender[train_idx]
                        y_g_eval = y_gender[eval_idx]

                    X_train, y_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_train)
                    X_eval, y_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_eval)

                    if(augmentation == 'true'):
                        X_train_flip = X_train[:, :, :, ::-1]
                        y_train_flip = y_train.copy()

                        X_train = np.concatenate((X_train, X_train_flip), axis=0)
                        y_train = np.concatenate((y_train, y_train_flip), axis=0)
                    
                    X_train, y_train = convert_tensor(X_train, y_train)
                    X_eval, y_eval = convert_tensor(X_eval, y_eval)

                    y_train = y_train.long()
                    y_eval = y_eval.long()

                    if(multi_task == 'true'):
                        _, y_g_train = convert_spectrograms(train_samples, conv_dim=conv_dim, sr=16000, labels=y_g_train)
                        _, y_g_eval = convert_spectrograms(eval_samples, conv_dim=conv_dim, sr=16000, labels=y_g_eval)

                        if(augmentation == 'true'):
                            y_g_train_flip = y_g_train.copy()
                            y_g_train = np.concatenate((y_g_train, y_g_train_flip))
                        
                        y_g_train = torch.tensor(y_g_train).float()
                        y_g_eval = torch.tensor(y_g_eval).float()

                        y_g_train = y_g_train.unsqueeze(-1)
                        y_g_eval = y_g_eval.unsqueeze(-1)

                    if(multi_task == 'true'):
                        train_ds = TensorDataset(X_train, y_train, y_g_train)
                        eval_ds = TensorDataset(X_eval, y_eval, y_g_eval)
                    else:
                        train_ds = TensorDataset(X_train, y_train)
                        eval_ds = TensorDataset(X_eval, y_eval)

                    train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)
                    eval_dataloader = DataLoader(eval_ds, batch_size=batch_size, num_workers=0, drop_last=True)

                    model = CLDNN(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                                  num_layers=num_layers, bidirectional=bidirectional,
                                  with_focus_attn=with_focus_attn).to(device)

                    if(multi_task == 'true'):
                        model_g = CLDNN_G(conv_dim=conv_dim, checkpoint=checkpoint, hidden_size=hidden_size,
                                            num_layers=num_layers, bidirectional=bidirectional,
                                            with_focus_attn=with_focus_attn).to(device)

                    if(multi_task == 'true'):
                        loss_func = nn.CrossEntropyLoss()
                        loss_func_g = nn.BCELoss()
                        optimizer = optim.Adam(list(model.parameters()) + list(model_g.parameters()), lr=learning_rate)
                    else:
                        loss_func = nn.CrossEntropyLoss()
                        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

                    if(use_warmup == 'true'):
                        t_total = len(train_dataloader) // 1 * num_epochs
                        opt_scheduler = WarmupLinearSchedule(optimizer, warmup_steps=t_total * 0.1, t_total=t_total)

                    train(train_dataloader, eval_dataloader, num_epochs)

                    model.eval()
                    if(multi_task == 'true'):
                        model_g.eval()

                    correct = 0
                    n = 0
                    for i in range(len(eval_samples)):
                        try:
                            X_new = preprocessing(eval_samples[i], method='mfcc', sr=16000, n_mfcc=n_mfcc)
                            X_new = convert_tensor(X_new).to(device)
                            y_new = model(X_new)
                            y_new = torch.argmax(nn.Softmax(dim=-1)(torch.mean(y_new, dim=0)))
                            #y_new = sorted(dict(collections.Counter(torch.argmax(nn.Softmax(dim=-1)(y_new), dim=1).cpu().numpy()))
                            #               .items(), key=(lambda x: x[1]), reverse=True)[0][0]
                            y_new = 1 if (y_new.item() == y[eval_idx][i].item()) else 0
                            correct += y_new
                            n += 1
                        except:
                            pass

                    acc = correct / n
                    acc_list.append(acc)

                acc_mean = sum(acc_list) / 5
                print('conv_dim:', conv_dim, '\taugmentation', augmentation, 
                      '\twith_focus_attn:', with_focus_attn, '\tmulti_task:', multi_task)
                print('Test accuray:', round(acc_mean, 5))
                print()